In [ ]:
# Data Source: https://www.kaggle.com/datasets/lakshmi25npathi/online-retail-dataset
# Folder: Online Retail Sales
# Description:
##This Online Retail II data set contains all the transactions occurring for a UK-based and registered, non-store online retail between 01/12/2009 and 09/12/2011.
##The company mainly sells unique all-occasion gift-ware. Many customers of the company are wholesalers.


# Analysis of Online retail sales dataset

## Overall goals:
- See the shape of the dataset and explore it
- Take a quick look at the data and decide on what to focus
- Manipulate data(changing data, creating new data, cleaning data etc...)
- Visualize findings to tell a story and back up analysis conducted
    - Visualizations include:
        - Geographical map
        - Bar Graph, Histogram, Scatterplot

## Libraries used throughout
- Pandas
- Plotly
- NumPy
- Matplotlib

### In the case of errors
- Not all python libraries may be on your machine and or within your directory. Ensure to install them.
- You ran a cell with an edit that you made to it(This notebook is designed to run seamlessly with no edits)
- Not running a python kernel or you're using an old version of python kernel
- Don't have libraries that are necessary for operation of parts or the entirety certain libraries.
    - Ex. nbformat is needed for certain features of the plotly library


In [ ]:
#Libraries to be used
# pip install "name of library"(incase there is an error where the library is requested or not identified by your system)
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px


In [ ]:
#convert the excel file to csv to create the main DataFrame
online_retail = pd.read_excel('Online_Retail.xlsx')
online_retail.to_csv('Online_Retail.csv', index=False)
online_retail.shape

In [ ]:
#Quick look of the DataFrame
online_retail.head()

### Attribute Information:
- InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
- StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.
- UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£).
- CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal. The name of the country where a customer resides.

### We will want to see what is unique about each of the qualitative columns
#### This will allow us to get an idea of distinction within the dataset where it matters

In [ ]:
#utilizing NumPy to find the count of unique values in the description column
description = online_retail['Description'].unique().tolist() #tolist isnt necessary but for safety it stores the array of values to a list
print(type(description))
len(description)


In [ ]:
cust_id = online_retail['CustomerID'].unique().tolist() #tolist isnt necessary but for safety it stores the array of values to a list
print(type(cust_id))
len(cust_id)


In [ ]:
country = online_retail['Country'].unique().tolist() #tolist isnt necessary but for safety it stores the array of values to a list
print(type(country))
len(country)


### We want to see if there are any null values within the data and analyze them to see whether they should be deleted or changed, or a mix of both

In [ ]:
online_retail[online_retail.isnull().any(axis=1)]

In [ ]:
#Interestingly 135080 rows/541909 rows have null values. There is a lot of cleaning to be done.
#Lets check what columns have null values
online_retail.isnull().any()

In [ ]:
#Here we can see that the focus is to be placed on the Description and CustomerID column.
#From this its possible to infer that maybe its orders that never went through or had errors. Lets check to see the counts of null values for each
online_retail['Description'].isnull().sum()


In [ ]:
#Same for customerID
online_retail['CustomerID'].isnull().sum()

## Overall more customerID rows are null in comparison to description
### Additionally, I've noticed that there are rows of data where the unit price is listed as 0.
- This isn't null, but to me I view it as that. Therefore, I will clean this data to remove rows with these conditions.

In [ ]:
#First find all the rows
cust_desc_errors = online_retail[online_retail['Description'].isnull() & online_retail['CustomerID'].isnull()]
cust_desc_errors


#new_online_retail = online_retail[online_retail[[cust_desc_errors & ]]]

In [ ]:
#Now we delete those rows which is essentially the opposite of our previous operation but we change from using & and use the or condition
new_online_retail = online_retail[online_retail['Description'].notnull() | online_retail['CustomerID'].notnull()]
new_online_retail

### We now have a slightly more cleaned up df which now has the issue with the null descriptions gone.
### We'll now sort the issue with the null customerID values. Note that its possible they may be guest purchases 

In [ ]:
new_online_retail.isnull().any()

In [ ]:
#Going forward, I'm gonna conduct some tests on other columns to find places where there may be errors like a negative quantity and also remove them as well
new_online_retail = new_online_retail[new_online_retail['Quantity'] > 0]
new_online_retail.shape

In [ ]:
#Next I'll check the unique values in each column that we arent sure of if there are valid values for
new_online_retail['Country'].unique()

In [ ]:
#Quick check on the uncspecified country to see if anything is wrong from first glance
new_online_retail[new_online_retail['Country'].str.contains('Unspecified')]

In [ ]:
#All the countries have no issues, however, there is EIRE which is actually Ireland and RSA which is actual South Africa.
#Therefore I will replace it in the df so it makes more sense to someone who checks the new df and any visualizations on countries
new_online_retail['Country'].replace({'EIRE': 'Ireland', 'RSA': 'South Africa'}, inplace=True) #inplace=True just modifies the df rather than creating a new one if it was false
new_online_retail

In [ ]:
#next I also want to check if there are any prices that are negative.
new_online_retail[new_online_retail['UnitPrice']<0]

In [ ]:
#We find that there is. And we do not want to keep this in our df.
#However we do want to keep it stored in a variable for future reference since it is a bad debt
bad_debt = new_online_retail['UnitPrice'] < 0
new_online_retail = new_online_retail.drop(new_online_retail[bad_debt].index)
new_online_retail.shape

### We have now cleaned up the dataset pretty well.
- Removed a few null values from various fields
- replaced the values of some fields to be more understandable to any audience
- Got rid of rows where the values didn't logically make sense

In [ ]:
#Again lets check what else has null values
new_online_retail.isnull().any()

In [ ]:
#How much more of these values are null?
new_online_retail[new_online_retail['CustomerID'].isnull()]

In [ ]:
import re  # library for regular expressions

# Find all unique stock codes
unique_stock_code = new_online_retail['StockCode'].drop_duplicates()

# Define the regular expression pattern
pattern = r'^[a-zA-Z\s]*$'

# Find stock codes with only words or letters
matching_vals = new_online_retail[new_online_retail['StockCode'].str.match(pattern, na=False)]
string_stock_codes = matching_vals['StockCode'].unique().tolist()
string_stock_codes

In [ ]:
# Let's analyze these further.
result_df = pd.DataFrame()  # Initialize an empty DataFrame to store results

for stock_code in string_stock_codes:
    subset_row = new_online_retail[new_online_retail['StockCode'] == stock_code].head(1)[['StockCode', 'Description']]
    result_df = pd.concat([result_df, subset_row], ignore_index=True)

# Display the result
result_df

In [ ]:
#Everything above looks fine except for the stockcodes of m and M
M_stock_codes = pd.DataFrame()

M_stock_codes = new_online_retail[new_online_retail['StockCode'] == 'M'].head(10)
M_stock_codes = pd.concat([M_stock_codes,new_online_retail[new_online_retail['StockCode'] == 'm'].head(10)],ignore_index=True)
M_stock_codes.head(20)


In [ ]:
#From this we were able to see two things, and it is that the little m was probably an error in the dataset
#That will get replaced as capital M.
#Also luckily there was only one occurence of this error
new_online_retail['StockCode'].replace('m','M', inplace=True)
M_stock_codes['StockCode'].replace('m','M', inplace=True)
M_stock_codes

## We have now cleaned up the dataset very well. And will now move on to analysis that we'll conduct on the countries of the dataset
### Some things to notice or consider are the following:
- The CustomerID column still contains null values, however there isn't anything that more we can do with it
    - The null values may just be guest purchases. Additionally, they are about 10,000 rows worth of data so it is not worth deleting
- We will first do some analysis on the US and then create a few visualizations to display findings
- We will also create a geographical map that showcases some densities in relation to where customers are from

### First is analysis of Sales in the US

#### We will start with creating a new df with only sales from customers in the US.

In [ ]:
#Lets check the unique countries within the new online retail sales df
new_online_retail['Country'].unique()


In [ ]:
#From the array, above we see US customers are listed as "USA"
#Now we create the new df
us_customers = new_online_retail[new_online_retail['Country'] == 'USA']
print(f"All transactions: {len(new_online_retail)} vs transactions from US customers: {len(us_customers)}")

In [ ]:
#So we can see that the US is a very small percentage of this retail store's us_customers
#From this, the owners of the retail store want to focus see two things below.
#What items are bought the most and how much money is spent on these items
#Lets look at the new df
us_customers.head(20)


In [ ]:
#Create a new column for the total money spent on an item

us_customers.insert(6,'Total spent',us_customers['Quantity'] * us_customers['UnitPrice'])
us_customers.head()

In [ ]:
#This will be used a few cells below but also serves a purpose here
unique_us_invoice = us_customers['InvoiceNo'].unique().tolist()
unique_us_invoice

In [ ]:
#Next we make a new df that will have the total price per order
us_invoice = us_customers[['InvoiceNo','Total spent', 'Country']].groupby(['InvoiceNo','Country']).agg({'Total spent': 'sum'}).reset_index()
us_invoice


In [ ]:
#list to hold all the values for the x-axis
i=0
X = []
while i < 5:
    X.append(f"US order #{i+1}")
    i += 1

#Y values
Y = us_invoice['Total spent'].tolist()
Y = [round(val,2) for val in Y]
Y


In [ ]:
#simple bar graph to showcase the different order sizes
plt.bar(X,Y)
plt.xlabel("US orders")
plt.ylabel("Money per order")
plt.yticks(Y,[f'${values}' for values in Y])
plt.title("US orders made")

plt.show()

In [ ]:
us_order1, us_order2, us_order3, us_order4, us_order5 = [us_customers[us_customers['InvoiceNo'] == val].reset_index(drop=True) for val in unique_us_invoice]
#example of one of the df with data for one unique invoiceNo
us_order1.head()


In [ ]:
us_order1.index.tolist()
us_order1['Total spent'].values

In [ ]:
# TODO: Fix the automation I have set for the graphs
#So this is a good first view of just how much was spent amongst the orders from customer(s) in the US.
#We see that there is decent variability and but most orders are around the range of $500 dollars excluding the extremely large purchase of 1579 total
#Next we want to see how the actual distribution of each unique item compares with each order and all orders together
fig, ((ax1, ax2, ax3), (ax4, ax5,ax6)) = plt.subplots(nrows=2, ncols=3, figsize=(20, 10))



#The subplots and dataframes are in lists that will be used within a for loop
graphs = [ax1,ax2,ax3,ax4,ax5]
orders = [us_order1, us_order2, us_order3, us_order4, us_order5]
colors=['red', 'green', 'blue', 'purple', 'orange']

#for loop to create all the scatter plots for comparisons
for graph, order,colour in zip(graphs, orders, colors):
    #common labels
    graph.set_xlabel('Order',fontsize=10)
    graph.set_ylabel('Total spent per item',fontsize=10)


    # Scatter plot
    graph.axis([-5, 200, -5, 100])
    graph.scatter(order.index.tolist(), order['Total spent'].values, color = colour)

    #adding dollar signs to the y ticks
    # Set the y-axis ticks with dollar signs
    y_ticks = graph.get_yticks()
    graph.set_yticklabels([f'${y}' for y in y_ticks])


#last graph that includes all orders together
ax6.set_xlabel('All orders',fontsize=10)
ax6.set_ylabel('Total spent per item',fontsize=10)
ax6.set_title('All orders together',fontsize = 15)

# scatter plot
ax6.axis([-5, 200, -5, 100])
ax6.scatter(us_customers.reset_index(drop=True).index.tolist(), us_customers['Total spent'].values, color = 'cyan')

#set y ticks again
ax6_yticks = ax6.get_yticks()
ax6.set_yticklabels(f'${a}' for a in ax6_yticks)

#Figure titling and then displaying everything
fig.suptitle("Comparison of distribution per unique items in each order", fontsize=20)
plt.show()


## We will finally end analysis and visualizations off with a map showcasing some information on customers/sales for all countries within the Dataset

In [ ]:
#Geographic data to create the map for the countries with choropleth library
country_geo = 'geo/world-countries.json'

In [ ]:
#check again for unique countries in the adjusted dataset one more time
new_online_retail['Country'].unique()

In [ ]:
#xtract the columns we need for the map, invoiceNo and Country
country_data = new_online_retail[['Country','InvoiceNo']]
country_data

In [ ]:

# Create a new DataFrame with unique pairs of 'Country' and 'InvoiceNo'
unique_country_data = country_data.drop_duplicates()


# Get the count of occurrences for each 'Country' across all 'InvoiceNo'
country_counts = unique_country_data['Country'].value_counts().reset_index()
country_counts.columns = ['Country', 'Count']

#adjusting unique_country_data to meet a field within the json data we will be using in the map a few cells down
#essentially USA is the abbreviation/id and not the actual name. Every other country uses its full name
country_counts['Country'].replace("USA","United States of America", inplace=True)
country_counts

In [ ]:
import plotly.express as px

# Interactive density map
fig = px.choropleth(
    country_counts,
    geojson=country_geo,
    locations='Country',
    featureidkey='properties.name',
    color='Count',
    color_continuous_scale='Viridis',
    range_color=(country_counts['Count'].min(), country_counts['Count'].max()),
    title='Pop Density of customers',
    labels={'Count': 'Customer Count'},
    width=1200,
    height=900
)

fig.show()

In [ ]:
#So we see that this graph, isnt very useful because the UK is a huge outlier given that the retail store is located and based in UK.
# Therefore we will create one last df that removes the UK and only focuses on every other country

every_other_country = country_counts[country_counts['Country'] != 'United Kingdom']
every_other_country

In [ ]:
# Interactive density map updated
fig = px.choropleth(
    every_other_country,
    geojson=country_geo,
    locations='Country',
    featureidkey='properties.name',
    color='Count',
    color_continuous_scale='Oranges', #used blues to keep the shading consistent. So darker means more customers
    range_color=(-20, every_other_country['Count'].max()),
    title='Pop Density of customers',
    labels={'Count': 'Customer Count'},
    width=1200,
    height=900
)

fig.show()

## This map is shows more variation amongst the countries even though there isn't as much outside of the countries that aren't neighbors to the UK except for a few. 
### Overall, the map allows for easy understanding of what countries that the UK retail store should or could focus sales, advertising, marketing etc... to